In [2]:
import pymssql
import pandas
import yaml

# CONFIG_FILE_PATH = 'config/config.yaml'
# config = yaml.load(open(CONFIG_FILE_PATH, 'r', encoding='utf-8'), Loader=yaml.FullLoader)

In [ ]:
def get_label_rank_df(
    model_id, # int
    dissimilarity_id, # int
    batch_size=100, # float
    relabel_lambda=0.069, # float
    random_ratio=0.5, # float
):
    
    batch_size_d = int((1- random_ratio)*batch_size)
    batch_size_r = batch_size - batch_size_d
    
    # call stored procedure 1 w/ D_ID
    args = {
        "M_ID": model_id,
        "D_ID": dissimilarity_id,
        "BATCH_SIZE": batch_size_d,
        "RELABEL_LAMBDA": relabel_lambda,
    }
    d_df = get_data_sp(sp="AL_RANKINGS", args)
    
    # call stored  procedure 1 w/ D_ID = 0 # represents random
    args['D_ID'] = 0
    args['BATCH_SIZE'] = batch_size_r
    r_df = get_data_sp(sp="AL_RANKINGS", args)
    
    full_df = pd.concat([d_df, r_df]) # might have duplicates but the chance is so low it doesn't matter
    
    return full_df
    

def get_data_sp(
    sp="", # string
    args, # dict
):
    # load from config file
    server = config['server']     # 'your_server.database.windows.net'
    database = config['database'] #'your_database'
    username = config['username'] # 'your_username'
    password = config['password'] #'your_password'

    # set up a connection to Azure
    with pymssql.connect(server, user, password, database) as conn:
        # set up a cursor object
        with conn.cursor(as_dict=True) as cursor:
            # gather variables for the stored procedure
            arg_tuples = (args['M_ID'], args['D_ID'], args['RELABEL_LAMBDA'], args['BATCH_SIZE'])
            # execute stored procedure
            cursor.callproc(sp, arg_tuples)
            
            # Fetch the results into a Pandas DataFrame
            df = pd.DataFrame(cursor.fetchall())
#             columns = [column[0] for column in cursor.description]
#             df = pd.DataFrame(cursor.fetchall(), columns=columns)
    return df
    
    

In [4]:
import sys
from pathlib import Path

# Get the path of the parent directory
parent_dir = str(Path.cwd().parent)

In [9]:
sys.path.append(parent_dir)

In [6]:
%load_ext autoreload
%autoreload 2



In [11]:
from importlib import reload

In [26]:
import sql_utils, sql_constants
reload(sql_utils)
reload(sql_constants)

<module 'sql_constants' from 'C:\\Users\\ysman\\Desktop\\Capstone\\PIVOT\\sql_constants.py'>

In [78]:
df = sql_utils.get_label_rank_df(model_id=1,dissimilarity_id=1,batch_size=5,)

In [85]:
sql_utils.get_test_set_df(model_id=1,minimum_percent=0.5)

,IMAGE_ID,PRED_LABEL,CONSENSUS
0,1,Label A,Confirmed Label A


In [84]:
sql_utils.run_sql_query(
    """
    Insert into labels (i_id, u_id, weight, date, label)
    Values (1,2, 1, '2024-01-27 12:00:00', 'Confirmed Label A' );
    """)

C:\Users\ysman\AppData\Local\Temp\ipykernel_11480\2592373361.py:1: UserWarning: The query returned empty.
  sql_utils.run_sql_query(


In [79]:
df

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,2,path/to/image2.jpg,0.6,Label B,0.8,0.455288
1,1,path/to/image1.jpg,0.5,Label A,0.9,0.435549
0,1,path/to/image1.jpg,1.0,Label A,0.9,0.871099
1,2,path/to/image2.jpg,1.0,Label B,0.8,0.758813


In [77]:
sql_utils.run_sql_query("select * from labels;")

,i_id,u_id,weight,date,label
0,1,1,1,2024-01-26 10:00:00,Confirmed Label A
1,2,1,2,2024-01-26 11:00:00,Confirmed Label B
2,1,2,1,2024-01-26 12:00:00,Confirmed Label C
3,2,2,2,2024-01-26 13:00:00,Confirmed Label D


In [76]:
sql_utils.run_sql_query("""
UPDATE labels
SET weight = weight / 10;
""")

C:\Users\ysman\AppData\Local\Temp\ipykernel_11480\1445388442.py:1: UserWarning: The query returned empty.
  sql_utils.run_sql_query("""


In [53]:
sql_utils.run_sql_query("select * from metrics;")

,i_id,m_id,d_id,d_value
0,1,0,0,1.0
1,2,0,0,1.0
2,3,0,0,1.0
3,6,0,0,1.0
4,7,0,0,1.0
5,1,1,1,0.5
6,2,1,1,0.6
7,1,2,2,0.4
8,2,2,2,0.3


In [13]:
sql_utils.run_sql_query("select * from metrics;")

,i_id,m_id,d_id,d_value
0,1,1,1,0.5
1,2,1,1,0.6
2,1,2,2,0.4
3,2,2,2,0.3


In [48]:
sql_utils.run_sql_query("select * from dissimilarity;")

,d_id,name,formula
0,0,random_sample,none
1,1,Dissimilarity Metric A,Formula A
2,2,Dissimilarity Metric B,Formula B


In [63]:
sql_utils.run_sql_query("select * from predictions;")

,m_id,i_id,class_prob,pred_label
0,1,1,0.9,Label A
1,1,2,0.8,Label B
2,2,1,0.7,Label C
3,2,2,0.6,Label D


In [47]:
sql_utils.run_sql_query(
"""
SET IDENTITY_INSERT dissimilarity ON;


INSERT INTO dissimilarity (d_id, name, formula)
VALUES (0, 'random_sample', 'none');
SET IDENTITY_INSERT dissimilarity OFF;
"""
)

In [45]:
try:
    print(hi)
except Exception as e:
    print(e.

<class 'NameError'>


In [71]:
a  = sql_utils.run_sql_query("""
WITH LABEL_COUNTS AS (
        SELECT
            I_ID,
            SUM(WEIGHT) AS W_COUNT
        FROM LABELS
        GROUP BY I_ID
    )
    SELECT
           I.I_ID AS IMAGE_ID,
           I.FILEPATH AS BLOB_FILEPATH,
           M.D_VALUE AS UNCERTAINTY,
           P.PRED_LABEL AS PRED_LABEL,
           P.CLASS_PROB AS PROBS,
           P.M_ID,
           M.M_ID,
           M.D_ID
          -- IIF(L.W_COUNT < 10, (M.D_VALUE * EXP(-0.069 * L.W_COUNT)), 0) AS RANK_SCORE
    FROM METRICS AS M
    INNER JOIN IMAGES AS I
        ON M.I_ID = I.I_ID
    INNER JOIN PREDICTIONS AS P
        ON M.I_ID = P.I_ID AND (M.M_ID = P.M_ID OR M.M_ID = 0) -- allow for test images that have M_ID=0
    INNER JOIN LABEL_COUNTS AS L
        ON M.I_ID = L.I_ID
    WHERE
          P.M_ID = 1
      AND M.D_ID = 0
    --ORDER BY RANK_SCORE DESC;
""")

check_fk = """
SELECT
    fk.name AS ForeignKeyName,
    tp.name AS ReferencedTable,
    ref_cols.name AS ReferencedColumn,
    tab.name AS TableName,
    fk_cols.name AS ColumnName
FROM
    sys.foreign_keys AS fk
JOIN
    sys.tables AS tab ON fk.parent_object_id = tab.object_id
JOIN
    sys.tables AS tp ON fk.referenced_object_id = tp.object_id
JOIN
    sys.foreign_key_columns AS fkc ON fkc.constraint_object_id = fk.object_id
JOIN
    sys.columns AS fk_cols ON fk_cols.object_id = fkc.parent_object_id AND fk_cols.column_id = fkc.parent_column_id
JOIN
    sys.columns AS ref_cols ON ref_cols.object_id = fkc.referenced_object_id AND ref_cols.column_id = fkc.referenced_column_id
WHERE
    fk.name = 'FK__metrics__m_id__0F624AF8';
"""
a

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,M_ID,M_ID,D_ID
0,1,path/to/image1.jpg,1.0,Label A,0.9,1,0,0
1,2,path/to/image2.jpg,1.0,Label B,0.8,1,0,0


In [62]:
a

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1,path/to/image1.jpg,1.0,Label C,0.7,0.0
1,2,path/to/image2.jpg,1.0,Label B,0.8,0.0


In [52]:
sql_utils.generate_random_evaluation_set(5)

In [111]:
sql_utils.execute_stored_procedure("AL_RANKINGS", args=args)

shit


,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1,path/to/image1.jpg,0.5,Label A,0.9,0.0
1,2,path/to/image2.jpg,0.6,Label B,0.8,0.0


In [114]:
if 1<= 2<=3:
    print("hi")

hi


In [15]:
sql_utils.create_alter_stored_procedure('GENERATE_RANDOM_TEST_SET')

Using preset file to create procedure GENERATE_RANDOM_TEST_SET: C:\Users\ysman\Desktop\Capstone\PIVOT\stored_procedures\Generate_Random_Test_Set.sql


In [ ]:
sql_utils.create_alter_stored_procedure('AL_RANKINGS')

In [80]:
sql_utils.create_alter_stored_procedure('MODEL_EVALUATION_MAX_CONSENSUS_FILTERING')

Using preset file to create procedure MODEL_EVALUATION_MAX_CONSENSUS_FILTERING: C:\Users\ysman\Desktop\Capstone\PIVOT\stored_procedures\Model_Evaluation_Filtering.sql


In [63]:
from collections import OrderedDict
args = OrderedDict([
    ("M_ID", 1),
    ("D_ID", 1),
    ("RELABEL_LAMBDA", 0.069),
    ("BATCH_SIZE", 3),
])

In [74]:
(if df) and (if df):

SyntaxError: invalid syntax (1904830851.py, line 1)

In [70]:
pd.concat([df, df],)

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,2,path/to/image2.jpg,0.6,Label B,0.8,0.0
1,1,path/to/image1.jpg,0.5,Label A,0.9,0.0
0,2,path/to/image2.jpg,0.6,Label B,0.8,0.0
1,1,path/to/image1.jpg,0.5,Label A,0.9,0.0


In [97]:
assert d

In [92]:
df

,IMAGE_ID,BLOB_FILEPATH,UNCERTAINTY,PRED_LABEL,PROBS,RANK_SCORE
0,1,path/to/image1.jpg,0.5,Label A,0.9,0.0
1,2,path/to/image2.jpg,0.6,Label B,0.8,0.0


In [41]:
check_procedures = """
    SELECT 
        name AS 'ProcedureName',
        create_date AS 'CreateDate',
        modify_date AS 'ModifyDate'
    FROM 
        sys.procedures;
"""

In [95]:
import pymssql

server = 'capstoneservercjault.database.windows.net'
user = 'CloudSA85da88d3'
password = 'Passwordtest123'
database = 'capstoneazure'


with pymssql.connect(server, user, password, database) as conn:
    with conn.cursor() as cursor:
        cursor.execute("""
        select * from Labels;
        """)
#         row = cursor.fetchone()
        row = cursor.fetchall()
#         while row:
#             print("SQL Server version:", row[0])
#             row = cursor.fetchone()
        columns = [column[0] for column in cursor.description]
        print(cursor.description)
df = pd.DataFrame(row, columns=columns)
df.head()

(('i_id', 3, None, None, None, None, None), ('u_id', 3, None, None, None, None, None), ('weight', 3, None, None, None, None, None), ('date', 4, None, None, None, None, None), ('label', 1, None, None, None, None, None))


,i_id,u_id,weight,date,label
0,1,1,10,2024-01-26 10:00:00,Confirmed Label A
1,2,1,20,2024-01-26 11:00:00,Confirmed Label B
2,1,2,15,2024-01-26 12:00:00,Confirmed Label C
3,2,2,25,2024-01-26 13:00:00,Confirmed Label D


In [27]:
df

,m_id,model_name,model_link,structure,weights
0,1,Model A,http://link-to-modela.com,JSON structure A,Binary data A
1,2,Model B,http://link-to-modelb.com,JSON structure B,Binary data B
